# Create a linear model in keras

In [1]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [16]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

## Create input and output for the model

In [ ]:
x = random((30, 2))

In [ ]:
y = np.dot(x, [2., 3.]) + 1

In [ ]:
x[:5]

In [ ]:
y[:5]

In [ ]:
lm = Sequential([Dense(1, input_shape=(2,))])
lm.compile(optimizer=SGD(lr=0.1), loss='mse')

### Check the preexisting loss function for x, y

In [ ]:
lm.evaluate(x,y)

### Fit the model to minimise the loss function for x and y using gradient descent

In [ ]:
lm.fit(x, y, nb_epoch=5, batch_size=1)

In [ ]:
lm.get_weights()

In [ ]:
lm.evaluate(x,y)

## Creating a linear model on predictions

First we like always create our untrained VGG model and get the data from the training and validation folders. Before that we set the paths obviously.

In [3]:
# path='data/dogscats/'
path='data/dogscats/sample/'
model_path=path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [4]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [5]:
#Setting the batch size to be large because we are crazy like that
batch_size=4

In [6]:
batches=get_batches(path+'train/', shuffle=False, batch_size=1)
val_batches=get_batches(path+'valid/', shuffle=False, batch_size=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


Using functions that concatenate all the batches so that we can save the entire array and dont have to proces the images everytime

In [ ]:
train_data = get_data(path+"train/")
val_data = get_data(path+'valid/')

Defining functions to save the arrays. We will be using the python library bcolz for that

In [8]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [ ]:
val_data.shape

In [ ]:
save_array(model_path+'train_data.bc', train_data)
save_array(model_path+'val_data.bc', val_data)

In [9]:
val_data=load_array(model_path+'val_data_sample.bc')
train_data=load_array(model_path+'train_data_sample.bc')

In [ ]:
val_data.classes()

Now we have to encode the classes of the VGG model

In [37]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1, 1)).todense())

In [38]:
trn_classes=batches.classes
val_classes=val_batches.classes
#Saving the classes
# save_array(model_path+'train_classes_sample.bc', trn_classes)
# save_array(model_path+'val_classes_sample.bc', val_classes)


In [39]:
# trn_classes=load_array(model_path+'train_classes_sample.bc')
# val_classes=load_array(model_path+'val_classes_sample.bc')
# #Now we one hot encode the classes
trn_labels=onehot(trn_classes)
val_labels=onehot(val_classes)

In [13]:
trn_features=model.predict(train_data, batch_size=4)
val_features=model.predict(val_data, batch_size=batch_size)

## Constructing the linear model

In [21]:
lm=Sequential([Dense(2, activation='softmax', input_shape=(1000,))])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
print(trn_labels)


[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [41]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 0s - loss: 0.5756 - acc: 0.6250 - val_loss: 0.5579 - val_acc: 0.7500
Epoch 2/3
16/16 [==============================] - 0s - loss: 0.3256 - acc: 1.0000 - val_loss: 0.5557 - val_acc: 0.7500
Epoch 3/3
16/16 [==============================] - 0s - loss: 0.2473 - acc: 1.0000 - val_loss: 0.5049 - val_acc: 0.7500


In [42]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 2)             2002        dense_input_2[0][0]              
Total params: 2002
____________________________________________________________________________________________________
